In [1]:
# Import dependencies
import pandas as pd
import chardet
# For data base creation
from sqlalchemy import create_engine
# from config import db_password
import os
import os.path, sys
import glob
import re

In [2]:
# tournament file location
file = '../../atp-world-tour-tennis-data/csv/1_tournaments/'

In [26]:
# def for cleaning tournament data
def clean_tournament(file):
    # assign path
    path, dirs, files = next(os.walk(file))
    file_count = len(files)
    # create empty list
    dataframes_list = []
    
    # append datasets to the list
    for i in range(file_count):
        with open(file+files[i], 'rb') as rawdata:
            result = chardet.detect(rawdata.read(100000))
        temp_df = pd.read_csv(file+files[i], encoding=result['encoding'], header=None)
        dataframes_list.append(temp_df)

    df = pd.concat(dataframes_list)
    df.drop(columns=[1,4,5,8,9,10,12,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30], inplace=True)
    df = df.reindex(columns=[0,2,3,7,6,11,13,14,22])
    header_df = pd.read_csv("../temp/Tourney_header.csv")
    df.columns = header_df.columns
    #Combinding coloumns 2,3
    df["Name"] = df["Name"]+"-"+ df["temp"]
    df = df.drop(columns =['temp'])
    df = df.reset_index()
    return df

In [27]:
# running CT function
df = clean_tournament(file)
df.head()

,index,Turney_id,Name,Date,Location,Draw_size,Conditions,Surface,Winner_id
0,0,1877-540,Grand Slam-Wimbledon,1877.07.09,"London, Great Britain",32,Outdoor,Grass,gi91
1,1,1878-540,Grand Slam-Wimbledon,1878.07.08,"London, Great Britain",64,Outdoor,Grass,hg50
2,2,1879-540,Grand Slam-Wimbledon,1879.07.07,"London, Great Britain",64,Outdoor,Grass,hg35
3,3,1880-540,Grand Slam-Wimbledon,1880.07.05,"London, Great Britain",64,Outdoor,Grass,hg35
4,4,1881-540,Grand Slam-Wimbledon,1881.07.02,"London, Great Britain",64,Outdoor,Grass,rg71


In [16]:
def clean_Match_Score(file):
    # finds information and puts it in a df
    file = '../../atp-world-tour-tennis-data/csv/2_match_scores/'

    # assign path
    path, dirs, files = next(os.walk(file))
    file_count = len(files)
    # create empty list
    dataframes_list = []
    
    # append datasets to the list
    for i in range(file_count):
        temp_df = pd.read_csv(file+files[i], encoding='utf-8', header=None)
        dataframes_list.append(temp_df)

    df_W = pd.concat(dataframes_list)
    df_L = pd.concat(dataframes_list)
    # dropping un needed winner stats
    df_W.drop(columns=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,21,22,23,24,25,26,28,29,30,31,32,33], inplace=True)
    # Dropping un needed Loser stats
    df_L.drop(columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,20,21,22,24,25,26,28,29,30,31,32,33,35], inplace=True)
    # reorder columns to match headers
    df_W =df_W.reindex(columns = [34,0,20,16,27,5])
    df_L=df_L.reindex(columns=[34,0,23,16,27,5])
    # setting winner&Loser headers
    df_W_Header = pd.read_csv("../temp/Match_score_header_W.csv")
    df_L_Header = pd.read_csv("../temp/Match_score_header_L.csv")
    # connecting winner&loser headers
    df_W.columns = df_W_Header.columns
    df_L.columns = df_L_Header.columns
    # Setting player_results
    df_W['Player_result'] = pd.Series('Won', index=df_L.index)
    df_L['Player_result'] = pd.Series('Lost', index=df_L.index)
    # Combining winner and loser dataframes
    df = pd.concat([df_W, df_L])
    df = df.reset_index()
    return df

In [23]:
df = clean_Match_Score(file)
temp = df['Score'].str.contains('65').sum() + df['Score'].str.contains('56').sum()
print(temp)
df.head()

512


,index,Match_id,Tourney_id,Player_id,Round,Score,Player_result
0,0,1877-540-NULL-4-1-gi91-mu62,1877-540,gi91,Finals,61 62 64,Won
1,1,1877-540-NULL-3-1-gi91-hg42,1877-540,gi91,Semi-Finals,62 65 62,Won
2,2,1877-540-NULL-2-3-gi91-lh23,1877-540,gi91,Quarter-Finals,63 62 56 61,Won
3,3,1877-540-NULL-2-2-hg42-mu66,1877-540,hg42,Quarter-Finals,63 63 65,Won
4,4,1877-540-NULL-2-1-mu62-e994,1877-540,mu62,Quarter-Finals,65 56 64 61,Won


In [25]:
# Creating a for loop to clean the Match_id column
# We need to remove the 3rd 4th and 5th word from the string
# First we need to remove the \r\n (-) from the string
# Then we need to split the string into a list
# Then we need to remove the 3rd 4th and 5th word from the list
# Then we need to rejoin the list into a string
# Then we need to reassign the string to the column

for i in range(1):
    _temp = df['Match_id'][i]
    print(_temp)
    _temp = _temp.replace('-', ' ')
    print(_temp)
    _temp = _temp[0] + '-' + _temp[1] + '-' + _temp[6]
    print(_temp)

1877-540-NULL-4-1-gi91-mu62
1877 540 NULL 4 1 gi91 mu62
1-8-4
